# The Battle of Neighborhoods

## Week 2 - Applied Data Science Capstone

In [1]:
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
from geopy.geocoders import Nominatim 
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import requests 
from pandas.io.json import json_normalize
!pip install folium
import folium
print('Libraries imported and installed.')

Libraries imported and installed.


In [2]:
CLIENT_ID = 'V0GEO1XYO5XM5OA4TA0RJJMR2W4TDYTCPM2HFA5ZK13QDOZO' # Deleted not to be posteb public on Git
CLIENT_SECRET = 'WKV3WIASC0VEBLMOYTZ1XK0GM2JY11UKC3RHLJ343STER4KE' # Deleted not to be posteb public on Git
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)

Your credentails:
CLIENT_ID: V0GEO1XYO5XM5OA4TA0RJJMR2W4TDYTCPM2HFA5ZK13QDOZO


In [3]:
LIMIT = 100
cities = ["Belgrade, Serbia", 'Zagreb, Croatia']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d123941735")
    results[city] = requests.get(url).json()

In [4]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

<ipython-input-4-7134362d7262>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  venues = json_normalize(results[city]['response']['groups'][0]['items'])


In [5]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=14)

    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='red',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of wine bars in {city} is ", results[city]['response']['totalResults'])

Total number of wine bars in Belgrade, Serbia is  51
Total number of wine bars in Zagreb, Croatia is  28


Here is the map of wine bars in Belgrade, Serbia:

In [6]:
maps[cities[0]]

Here is the map of wine bars in Zagreb, Croata:

In [7]:
maps[cities[1]]

By looking on the maps above we can see that density of wine bars is bigger in Zagreb. Let`s calculate to prove it:

In [8]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=14)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
  
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='red',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))
    print("____________________________________") 

Belgrade, Serbia
Mean Distance from Mean coordinates
0.020989751460307772
____________________________________
Zagreb, Croatia
Mean Distance from Mean coordinates
0.02231054527921986
____________________________________


Let`s visualise it:

In [9]:
maps[cities[0]]

In [10]:
maps[cities[1]]